In [1]:
import numpy as np
import trimesh
from tqdm import tqdm

In [2]:
def uniform_sampling_from_mesh(vertices, faces, sample_num):
    # -------- TODO -----------
    # 1. compute area of each triangles
    # 2. compute probability of each triangles from areas
    # 3. sample N faces according to the probability
    # 4. for each face, sample 1 point
    # Note that FOR-LOOP is not allowed!
    # -------- TODO -----------
    
    vec1 = vertices[faces[:, 1]] - vertices[faces[:, 0]]
    vec2 = vertices[faces[:, 2]] - vertices[faces[:, 0]]
    area = np.linalg.norm(np.cross(vec1, vec2), axis=1) / 2
    
    prob = area / np.sum(area)
    
    cum_prob = np.cumsum(prob)
    uniform_pc = np.random.rand(sample_num)
    face_idx = np.searchsorted(cum_prob, uniform_pc)
    
    r1 = np.random.uniform(0, 1, sample_num).reshape(-1, 1)
    r2 = np.random.uniform(0, 1, sample_num).reshape(-1, 1)
    r1_sqrt = np.sqrt(r1).reshape(-1, 1)
    uniform_pc = (1 - r1_sqrt) * vertices[faces[face_idx, 0]] + r1_sqrt * (1 - r2) * vertices[faces[face_idx, 1]] + r1_sqrt * r2 * vertices[faces[face_idx, 2]]
    
    return area, prob, uniform_pc
        

In [3]:
def farthest_point_sampling(pc, sample_num):
    # -------- TODO -----------
    # FOR LOOP is allowed here.
    # -------- TODO -----------

    N = pc.shape[0]
    for i in tqdm(range(1, sample_num), desc="iteration"):
        selected_list = np.repeat(pc[:i, :].reshape(1, -1, 3), N - i, axis=0)
        candidate_list = np.repeat(pc[i:, :].reshape(-1, 1, 3), i, axis=1)
        dist = selected_list - candidate_list
        dist = np.linalg.norm(dist, axis=2)
        dist = np.min(dist, axis=1)
        idx = np.argmax(dist)
        pc[i, :], pc[i + idx, :] = pc[i + idx, :], pc[i, :]

    return pc[:sample_num]

In [4]:
# task 1: uniform sampling

obj_path = 'spot.obj'
mesh = trimesh.load(obj_path)
print('faces shape: ', mesh.faces.shape)
sample_num = 512
area, prob, uniform_pc = uniform_sampling_from_mesh(mesh.vertices, mesh.faces, sample_num)

# Visualization. For you to check your code
np.savetxt('uniform_sampling_vis.txt', uniform_pc)

print('area shape: ',area.shape)
print('prob shape: ',prob.shape)
print('pc shape: ',uniform_pc.shape)
# the result should satisfy:
#       area.shape = (13712, )
#       prob.shape = (13712, )
#       uniform_pc.shape = (512, 3)

# For submission
save_dict = {'area': area, 'prob': prob, 'pc': uniform_pc}
np.save('../results/uniform_sampling_results', save_dict)

trimesh.Scene([
    trimesh.PointCloud(uniform_pc, colors=[0, 0, 0, 1])
]).show()

faces shape:  (13712, 3)
area shape:  (13712,)
prob shape:  (13712,)
pc shape:  (512, 3)


In [5]:
# task 2: FPS

init_sample_num = 2000
final_sample_num = 512
_,_, tmp_pc = uniform_sampling_from_mesh(mesh.vertices, mesh.faces, init_sample_num)
fps_pc = farthest_point_sampling(tmp_pc, final_sample_num)

# Visualization. For you to check your code
np.savetxt('fps_vis.txt', fps_pc)

# For submission
np.save('../results/fps_results', fps_pc)

trimesh.Scene([
    trimesh.PointCloud(fps_pc, colors=[0, 0, 0, 1]),
]).show()

iteration: 100%|██████████| 511/511 [00:09<00:00, 51.73it/s] 


In [6]:
# task 3: metrics

from earthmover import earthmover_distance   # EMD may be very slow (1~2mins)
# -----------TODO---------------
# compute chamfer distance and EMD for two point clouds sampled by uniform sampling and FPS.
# sample and compute CD and EMD again. repeat for five times.
# save the mean and var.
# -----------TODO---------------

def chamfer_distance(pc1, pc2):
    pc1 = np.repeat(pc1.reshape(1, -1, 3), pc2.shape[0], axis=0)
    pc2 = np.repeat(pc2.reshape(-1, 1, 3), pc1.shape[1], axis=1)
    dist = pc1 - pc2
    dist = np.linalg.norm(dist, axis=2)
    dist1 = np.min(dist, axis=1)
    dist2 = np.min(dist, axis=0)
    return np.mean(dist1) + np.mean(dist2)

CD_error = []
EMD_error = []

for _ in range(5):
    _, _, uniform_pc = uniform_sampling_from_mesh(mesh.vertices, mesh.faces, sample_num)
    _, _, fps_pc = uniform_sampling_from_mesh(mesh.vertices, mesh.faces, init_sample_num)
    fps_pc = farthest_point_sampling(fps_pc, final_sample_num)
    CD_error.append(chamfer_distance(uniform_pc, fps_pc))
    EMD_error.append(earthmover_distance(uniform_pc, fps_pc))
    
CD_mean = np.mean(CD_error)
CD_var = np.var(CD_error)
EMD_mean = np.mean(EMD_error)
EMD_var = np.var(EMD_error)

print('CD_mean: ', CD_mean)
print('CD_var: ', CD_var)
print('EMD_mean: ', EMD_mean)
print('EMD_var: ', EMD_var)

# For submission
np.save('../results/metrics', {'CD_mean':CD_mean, 'CD_var':CD_var, 'EMD_mean':EMD_mean, 'EMD_var':EMD_var})

iteration: 100%|██████████| 511/511 [00:10<00:00, 47.83it/s] 


move 0.001953125 dirt from [32.02018804 28.99330244 41.46983311] to [34.56694466 28.05924095 41.73930658] for a cost of 0.005324211636100604
move 0.001953125 dirt from [17.36521454 34.03520557 23.8346878 ] to [18.5871934  33.10119888 25.49165456] for a cost of 0.004415591758023794
move 0.001953125 dirt from [24.78744783 25.7781937  41.24756024] to [25.83207113 24.09311398 41.94266604] for a cost of 0.00410337738895021
move 0.001953125 dirt from [28.04178922 33.57376582 38.84486277] to [29.03610152 33.62758584 39.11089011] for a cost of 0.002013068507219283
move 0.001953125 dirt from [ 7.18167035 34.95828626 37.7716034 ] to [10.93097892 31.53501544 38.00822405] for a cost of 0.009926811788532643
move 0.001953125 dirt from [14.67358211 30.5453121  34.37345655] to [19.54716736 25.07977086 34.17196977] for a cost of 0.014307833265733682
move 0.001953125 dirt from [39.09008519 34.49411301 27.41278927] to [39.28364609 34.29034238 27.05132789] for a cost of 0.0008942724015047794
move 0.001953

iteration: 100%|██████████| 511/511 [00:09<00:00, 56.14it/s] 


move 0.001953125 dirt from [11.68466277 36.16319869 21.82902853] to [10.83772138 35.59361765 21.96076144] for a cost of 0.0020099979659833934
move 0.001953125 dirt from [22.02080357 13.88882698 24.27561354] to [22.92128398 13.17041266 21.87339573] for a cost of 0.0052033957193565235
move 0.001953125 dirt from [ 6.76873121 34.71855047 34.14009859] to [ 7.07355665 34.37396383 35.73174257] for a cost of 0.003235939159081011
move 0.001953125 dirt from [46.71435688 13.17796835 29.74587208] to [46.02960582 12.37897293 29.3866655 ] for a cost of 0.002171666917976607
move 0.001953125 dirt from [19.44759081 37.09633547 40.00456457] to [20.71356846 37.06795544 39.46091602] for a cost of 0.002691529980484779
move 0.001953125 dirt from [48.88050609 31.50592462 25.91501915] to [47.35673947 32.07703098 25.76259909] for a cost of 0.0031921849273049137
move 0.001953125 dirt from [12.67212913 38.01394182 41.00150173] to [13.07286839 40.58025398 40.32727859] for a cost of 0.005241194436562864
move 0.001

iteration: 100%|██████████| 511/511 [00:08<00:00, 57.75it/s] 


move 0.001953125 dirt from [44.68303104  9.4795614  35.63136517] to [45.86376381  9.07294764 34.50257811] for a cost of 0.003287768395862829
move 0.001953125 dirt from [21.94616767 19.67085953 36.57197232] to [22.49973308 16.84300732 36.16526877] for a cost of 0.005683758368562008
move 0.001953125 dirt from [41.60515787 30.58076531 23.34580471] to [39.95615568 30.78831787 23.40406197] for a cost of 0.0032481121781800133
move 0.001953125 dirt from [51.56694149 16.13337749 23.17717292] to [50.08801139 18.18717587 22.0639068 ] for a cost of 0.005400199378661081
move 0.001953125 dirt from [15.80585409 48.11396681 40.32393959] to [15.13403735 47.21357355 39.47776895] for a cost of 0.0027469371785337495
move 0.001953125 dirt from [32.31486147 30.90074245 40.54695781] to [31.23536057 30.71049879 40.71130019] for a cost of 0.0021648196935592765
move 0.001953125 dirt from [26.85382862  9.60602499 21.3927102 ] to [27.15009605  8.69091441 21.99404069] for a cost of 0.0022155705661326733
move 0.00

iteration: 100%|██████████| 511/511 [00:10<00:00, 48.79it/s] 


move 0.001953125 dirt from [ 8.84139798 41.51468547 28.55201103] to [ 9.72718547 43.17869098 28.2911924 ] for a cost of 0.0037168744504121123
move 0.001953125 dirt from [22.62771143 27.46464083 23.92743793] to [23.65870512 26.9605551  22.73228655] for a cost of 0.003236203316570805
move 0.001953125 dirt from [53.33108099 25.37293364 35.42548881] to [52.76988882 22.83322667 36.42875493] for a cost of 0.005444838572909304
move 0.001953125 dirt from [15.53680377 46.97446889 40.82163899] to [17.53389451 51.50685206 38.03906055] for a cost of 0.011095677410787068
move 0.001953125 dirt from [44.01112796 31.88233812 38.25520545] to [45.69038676 29.74612618 39.83741886] for a cost of 0.006141238364374711
move 0.001953125 dirt from [16.07043216 51.57617347 25.75320139] to [16.25324189 52.96814471 25.31975049] for a cost of 0.00286975350016016
move 0.001953125 dirt from [48.96711088 21.2772085  22.10595384] to [48.47224369 23.91049702 22.06333875] for a cost of 0.0052338349628268015
move 0.00195

iteration: 100%|██████████| 511/511 [00:09<00:00, 54.29it/s] 


move 0.001953125 dirt from [35.57607898 36.07374967 35.33473704] to [24.73107367 45.04575982 35.3279641 ] for a cost of 0.027490617871483495
move 0.001953125 dirt from [39.01190971 14.75174809 25.43237524] to [36.66825944 13.98517033 26.36380042] for a cost of 0.005148213310229164
move 0.001953125 dirt from [43.77163518 11.86603333 37.76628687] to [44.66540703 16.82370862 41.24440781] for a cost of 0.011956363455999717
move 0.001953125 dirt from [37.45237352 34.94328501 35.79641923] to [35.8815531  35.49807843 36.17121727] for a cost of 0.003335069508940982
move 0.001953125 dirt from [ 7.12162507 34.27143441 27.88504121] to [ 6.74648976 36.31003672 26.04183725] for a cost of 0.0054175993438512595
move 0.001953125 dirt from [26.60938541 40.37836019 27.39447049] to [25.65106085 43.29703482 27.14761422] for a cost of 0.006019297250727631
move 0.001953125 dirt from [26.17954277 13.44001846 29.1346056 ] to [26.56172619 13.00192676 29.08595338] for a cost of 0.0011394530849957922
move 0.0019